# MVP workflow
- Data from clinicaltrials.gov

In [1]:
import json, os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import scispacy

/Users/cmserna/Sites/virtualenvs/clinical-trials/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Load json file with parsed information

In [2]:
# paths
path_to_json_file = os.path.abspath('../data/json/')

# json file
json_file = '/all_parsed_data_json' #name json file  
file = '{}{}.json'.format(path_to_json_file, json_file)

# load json file
df = pd.read_json(file)

In [3]:
df.head()

,nct_id,study_first_submitted,source,brief_title,overall_status,verification_date,study_type,study_first_posted,last_update_submitted,last_update_posted,...,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose
0,NCT01236404,"November 5, 2010",PhaseBio Pharmaceuticals Inc.,"Phase 1/2a, Randomized, Double-Blind, Placebo-...",Completed,May 2013,Interventional,"November 8, 2010","May 13, 2013","May 21, 2013",...,None,\n Primary objective:\n\n To evaluat...,Walnut Creek,United States,94598,PhaseBio Pharmaceuticals Inc.,Industry,Randomized,Parallel Assignment,Treatment
1,NCT00275600,"January 10, 2006",Mayo Clinic,Randomized Clinical Trial of Vitamin E and Eve...,Completed,November 2009,Interventional,"January 12, 2006","November 25, 2009","November 26, 2009",...,\n Cyclical mastalgia is a common complai...,\n This study is being done to find out w...,Rochester,United States,55905,Mayo Clinic,Other,Randomized,Parallel Assignment,Treatment
10,NCT01810211,"March 8, 2013",Nova Southeastern University,The Most Effective Intervention for Reducing P...,Completed,March 2014,Interventional,"March 13, 2013","September 24, 2014","September 26, 2014",...,None,"\n This study will look at which, if any,...",Raleigh,United States,27609,Nova Southeastern University,Other,Randomized,Parallel Assignment,Treatment
100,NCT00826111,"January 19, 2009",Steward St. Elizabeth's Medical Center of Bost...,The Effects of Eszopiclone and Lexapro on Pref...,Completed,June 2012,Interventional,"January 21, 2009","June 28, 2012","June 29, 2012",...,None,\n The study examined the effects of addi...,Boston,United States,02135,Steward St. Elizabeth's Medical Center of Bost...,Other,Randomized,Parallel Assignment,Treatment
1000,NCT02813850,"June 23, 2016",Assistance Publique - Hôpitaux de Paris,Oxygen Therapy and Pregnancy in Sickle Cell Di...,Recruiting,March 2017,Interventional,"June 27, 2016","December 20, 2017","December 21, 2017",...,\n Sickle cell disease (SCD) corresponds ...,\n The purpose of this study is to assess...,Paris,France,75015,Assistance Publique - Hôpitaux de Paris,Other,Randomized,Parallel Assignment,Prevention


In [4]:
df.shape

(5984, 24)

## Preprocess and clean data

In [5]:
# change column type
def data_types(dataframe, cols = [], to_type = ''):
    for col in cols:
        dataframe[col] = df[col].astype(to_type)

In [6]:
columns_dates = ['study_first_submitted', 'study_first_posted', 'last_update_submitted', 'last_update_posted', 'verification_date']
data_types(df, columns_dates, 'datetime64')

In [7]:
# Extract year 
df['year_submitted'] = df['study_first_submitted'].dt.year
df['year_last_updated'] = df['last_update_submitted'].dt.year

In [8]:
# Remove /n in all df
df = df.replace(r'\n',' ', regex=True)

In [9]:
# Create new column for all text
df['all_text'] = df['source'] + ' ' + df['brief_title'] + ' ' + df['condition'] + ' ' + df['condition_browse/mesh_term'] + ' '+ df['intervention_browse/mesh_term'] + ' '+ df['detailed_description/textblock'] + ' ' + df['brief_summary/textblock']

# All_text in lowercase
df['all_text'] = df['all_text'].str.lower()

# remove extra whitespace
df.all_text = df.all_text.replace('\s+', ' ', regex=True)


In [10]:
df['all_text'][0]

'phasebio pharmaceuticals inc. phase 1/2a, randomized, double-blind, placebo-controlled, study to assess safety, tolerability, pk and pd response of pb1023 injection following single and multiple sq doses in adults with type 2 diabetes mellitus diabetes mellitus, type 2 diabetes mellitus none none primary objective: to evaluate the safety and tolerability of single and multiple ascending doses of pb1023 administered as a subcutaneous (sc) injection in adult subjects with t2dm. secondary objectives: 1. to characterize the pharmacokinetic profile of pb1023 after single and multiple ascending doses of pb1023. 2. to assess the pharmacodynamic response of various single and multiple doses of pb1023 (daily fasting plasma glucose, and serial glucose, c-peptide and insulin levels in response to a liquid mixed meal tolerance test (mmtt). '

In [11]:
# Remove extra white space in summary

df['brief_summary/textblock'] = df['brief_summary/textblock'].replace('\s+', ' ', regex=True)
df['brief_summary/textblock'][0]


' Primary objective: To evaluate the safety and tolerability of single and multiple ascending doses of PB1023 administered as a subcutaneous (SC) injection in adult subjects with T2DM. Secondary objectives: 1. To characterize the pharmacokinetic profile of PB1023 after single and multiple ascending doses of PB1023. 2. To assess the pharmacodynamic response of various single and multiple doses of PB1023 (daily fasting plasma glucose, and serial glucose, c-peptide and insulin levels in response to a liquid Mixed Meal Tolerance Test (MMTT). '

## ML preprocessing
- Lowercasing, lemmatization, and stop-word removal
- TD-IDF
- Test different methods for classification

In [31]:
# Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [32]:
pat_numbers = lambda x: re.sub(r'(\d)+', '', x.lower())

cv = CountVectorizer(stop_words='english',
                     preprocessor = pat_numbers,
                     max_features = 3000,
                     lowercase = True,
                     max_df = 0.5,
                     ngram_range = (1, 3))

In [33]:
docs = list(df['all_text'])

X = cv.fit_transform(docs)

In [34]:
len(cv.vocabulary_)

3000

In [37]:
pd.DataFrame(X.toarray(), columns=cv.get_feature_names()).head()

,abdominal,ability,ablation,able,abnormal,abnormalities,absence,absence disease,absence disease progression,absorption,...,year,years,years age,years old,york,young,young adults,youth,zinc,µg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF - IDF

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer

In [56]:
transformer = TfidfTransformer()
tweights = transformer.fit_transform(X)
tweights

<5984x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 480733 stored elements in Compressed Sparse Row format>

In [57]:
# turn weights data into a dataframe
tf = pd.DataFrame(tweights.toarray(), columns=cv.get_feature_names())

In [58]:
# Top terms by average tf-idf weight
weights = np.asarray(tweights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cv.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
2813,treatment,0.031298
337,cancer,0.023154
2402,safety,0.020140
2663,subjects,0.019977
763,disease,0.018786
791,dose,0.018533
2761,therapy,0.016768
1871,pain,0.016459
839,efficacy,0.016299
446,clinical,0.015944


In [59]:
tf.sample(5)

abdominal  ability  ablation      able  abnormal  abnormalities  \
2489   0.000000      0.0       0.0  0.000000       0.0            0.0   
2519   0.000000      0.0       0.0  0.030811       0.0            0.0   
2175   0.041521      0.0       0.0  0.000000       0.0            0.0   
4389   0.000000      0.0       0.0  0.000000       0.0            0.0   
2196   0.000000      0.0       0.0  0.000000       0.0            0.0   

      absence  absence disease  absence disease progression  absorption  \
2489      0.0              0.0                          0.0         0.0   
2519      0.0              0.0                          0.0         0.0   
2175      0.0              0.0                          0.0         0.0   
4389      0.0              0.0                          0.0         0.0   
2196      0.0              0.0                          0.0         0.0   

      abstinence  abuse  acceptability  acceptable  accepted    access  \
2489         0.0    0.0            0.0         0.0       0.0  0.000000   
2519         0.0    0.0            0.0         0.0       0.0  0.032907   
2175         0.0    0.0            0.0         0.0       0.0  0.000000   
4389         0.0    0.0            0.0         0.0       0.0  0.000000   
2196         0.0    0.0            0.0         0.0       0.0  0.000000   

      according  account  accrual  accrual total  accrual total patients  \
2489        0.0      0.0      0.0            0.0                     0.0   
2519        0.0      0.0      0.0            0.0                     0.0   
2175        0.0      0.0      0.0            0.0                     0.0   
4389        0.0      0.0      0.0            0.0                     0.0   
2196        0.0      0.0      0.0            0.0                     0.0   

      accrued  accrued study  accuracy  accurate  acetate   achieve  achieved  \
2489      0.0            0.0       0.0       0.0      0.0  0.000000       0.0   
2519      0.0            0.0       0.0       0.0      0.0  0.033102       0.0   
2175      0.0            0.0       0.0       0.0      0.0  0.000000       0.0   
4389      0.0            0.0       0.0       0.0      0.0  0.000000       0.0   
2196      0.0            0.0       0.0       0.0      0.0  0.000000       0.0   

      achieving  acid  acids  acne  acquired  acquisition       act   acting  \
2489        0.0   0.0    0.0   0.0       0.0          0.0  0.000000  0.00000   
2519        0.0   0.0    0.0   0.0       0.0          0.0  0.041577  0.04339   
2175        0.0   0.0    0.0   0.0       0.0          0.0  0.000000  0.00000   
4389        0.0   0.0    0.0   0.0       0.0          0.0  0.000000  0.00000   
2196        0.0   0.0    0.0   0.0       0.0          0.0  0.000000  0.00000   

      action  activated  activation  active  activities  activity  actual  \
2489     0.0        0.0         0.0     0.0         0.0       0.0     0.0   
2519     0.0        0.0         0.0     0.0         0.0       0.0     0.0   
2175     0.0        0.0         0.0     0.0         0.0       0.0     0.0   
4389     0.0        0.0         0.0     0.0         0.0       0.0     0.0   
2196     0.0        0.0         0.0     0.0         0.0       0.0     0.0   

      acupuncture  acute  acute myeloid  acute myeloid leukemia   ad  \
2489          0.0    0.0            0.0                     0.0  0.0   
2519          0.0    0.0            0.0                     0.0  0.0   
2175          0.0    0.0            0.0                     0.0  0.0   
4389          0.0    0.0            0.0                     0.0  0.0   
2196          0.0    0.0            0.0                     0.0  0.0   

      adalimumab  adapted  add  added  adding  addition  additional  \
2489         0.0      0.0  0.0    0.0     0.0  0.000000         0.0   
2519         0.0      0.0  0.0    0.0     0.0  0.000000         0.0   
2175         0.0      0.0  0.0    0.0     0.0  0.000000         0.0   
4389         0.0      0.0  0.0    0.0     0.0  0.000000         0.

In [60]:
# Create new dataframe with nct-id and merge wth tf by index
pd.set_option('display.max_columns', None) 

df_tf = df['nct_id']
df_tf.head()

0       NCT01236404
1       NCT00275600
10      NCT01810211
100     NCT00826111
1000    NCT02813850
Name: nct_id, dtype: object

In [61]:
df_tf = pd.merge(df_tf, tf, left_index=True, right_index=True)

In [62]:
df_tf.head()

nct_id  abdominal  ability  ablation  able  abnormal  \
0     NCT01236404        0.0      0.0       0.0   0.0       0.0   
1     NCT00275600        0.0      0.0       0.0   0.0       0.0   
10    NCT01810211        0.0      0.0       0.0   0.0       0.0   
100   NCT00826111        0.0      0.0       0.0   0.0       0.0   
1000  NCT02813850        0.0      0.0       0.0   0.0       0.0   

      abnormalities  absence  absence disease  absence disease progression  \
0               0.0      0.0              0.0                          0.0   
1               0.0      0.0              0.0                          0.0   
10              0.0      0.0              0.0                          0.0   
100             0.0      0.0              0.0                          0.0   
1000            0.0      0.0              0.0                          0.0   

      absorption  abstinence  abuse  acceptability  acceptable  accepted  \
0            0.0         0.0    0.0            0.0    0.000000       0.0   
1            0.0         0.0    0.0            0.0    0.077117       0.0   
10           0.0         0.0    0.0            0.0    0.000000       0.0   
100          0.0         0.0    0.0            0.0    0.000000       0.0   
1000         0.0         0.0    0.0            0.0    0.000000       0.0   

      access  according  account  accrual  accrual total  \
0        0.0        0.0      0.0      0.0            0.0   
1        0.0        0.0      0.0      0.0            0.0   
10       0.0        0.0      0.0      0.0            0.0   
100      0.0        0.0      0.0      0.0            0.0   
1000     0.0        0.0      0.0      0.0            0.0   

      accrual total patients  accrued  accrued study  accuracy  accurate  \
0                        0.0      0.0            0.0       0.0       0.0   
1                        0.0      0.0            0.0       0.0       0.0   
10                       0.0      0.0            0.0       0.0       0.0   
100                      0.0      0.0            0.0       0.0       0.0   
1000                     0.0      0.0            0.0       0.0       0.0   

      acetate  achieve  achieved  achieving  acid  acids  acne  acquired  \
0         0.0      0.0       0.0        0.0   0.0    0.0   0.0       0.0   
1         0.0      0.0       0.0        0.0   0.0    0.0   0.0       0.0   
10        0.0      0.0       0.0        0.0   0.0    0.0   0.0       0.0   
100       0.0      0.0       0.0        0.0   0.0    0.0   0.0       0.0   
1000      0.0      0.0       0.0        0.0   0.0    0.0   0.0       0.0   

      acquisition  act  acting  action  activated  activation  active  \
0             0.0  0.0     0.0     0.0   0.000000         0.0     0.0   
1             0.0  0.0     0.0     0.0   0.000000         0.0     0.0   
10            0.0  0.0     0.0     0.0   0.000000         0.0     0.0   
100           0.0  0.0     0.0     0.0   0.000000         0.0     0.0   
1000          0.0  0.0     0.0     0.0   0.091838         0.0     0.0   

      activities  activity  actual  acupuncture  acute  acute myeloid  \
0            0.0       0.0     0.0          0.0    0.0            0.0   
1            0.0       0.0     0.0          0.0    0.0            0.0   
10           0.0       0.0     0.0          0.0    0.0            0.0   
100          0.0       0.0     0.0          0.0    0.0            0.0   
1000         0.0       0.0     0.0          0.0    0.0            0.0   

      acute myeloid leukemia   ad  adalimumab  adapted  add  added  adding  \
0                        0.0  0.0         0.0      0.0  0.0    0.0     0.0   
1                        0.0  0.0         0.0      0.0  0.0    0.0     0.0   
10                       0.0  0.0         0.0      0.0  0.0    0.0     0.0   
100                      0.0  0.0         0.0      0.0  0.0    0.0     0.0   
1000                     0.0  0.0         0.0      0.0  0.0    0.0     0.0   

      addition  additional  additionally  address  adenocarc